In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

algousd = yf.download('ALGO-USD', start='2025-01-01', end=datetime.now(), progress=False, auto_adjust=True)
# Handle multi-level columns from yfinance
if isinstance(algousd.columns, pd.MultiIndex):
    algousd.columns = algousd.columns.get_level_values(0)

# Create a DataFrame with just the close prices
price_df = pd.DataFrame({
    'date': algousd.index,
    'open': algousd['Open'].values,
    'high': algousd['High'].values,
    'low': algousd['Low'].values,
    'close': algousd['Close'].values,
    'volume': algousd['Volume'].values
})

price_df['date'] = pd.to_datetime(price_df['date'])
price_df['quarter'] = price_df['date'].dt.to_period('Q')
price_df['avg_price'] = (price_df['high'] + price_df['low'] + price_df['close'] + price_df['open']) / 4

price_df['pv'] = price_df['avg_price'] * price_df['volume']
price_df['cumulative_pv'] = price_df.groupby('quarter')['pv'].cumsum()
price_df['cumulative_volume'] = price_df.groupby('quarter')['volume'].cumsum()

# Calculate VWAP
price_df['vwap'] = price_df['cumulative_pv'] / price_df['cumulative_volume']


price_df_quarter_ends = price_df[pd.to_datetime(price_df['date']).dt.is_quarter_end]
price_df_quarter_ends



,date,open,high,low,close,volume,quarter,avg_price,pv,cumulative_pv,cumulative_volume,vwap
89,2025-03-31,0.179389,0.181528,0.172695,0.180097,70773544,2025Q1,0.178427,1.262793e+07,5.600097e+09,16610081996,0.337150
180,2025-06-30,0.185949,0.186967,0.177142,0.185459,64516672,2025Q2,0.183879,1.186328e+07,1.433571e+09,7152846809,0.200420
272,2025-09-30,0.207763,0.210226,0.201663,0.207899,107192472,2025Q3,0.206888,2.217681e+07,2.908630e+09,11428128235,0.254515
